In [123]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('sentiment140/training.1600000.processed.noemoticon.csv',index_col=None, header=None, encoding='latin-1')

In [124]:
g = df.groupby(by = df[4]).groups

In [128]:
df.iloc[g['000catnap000']]

,0,1,2,3,4,5
710344,0,2257881753,Sat Jun 20 15:02:40 PDT 2009,NO_QUERY,000catnap000,kinda miss my mom shouldn't have called her t...
849111,4,1565014948,Mon Apr 20 05:39:01 PDT 2009,NO_QUERY,000catnap000,@Wumbologist huh uhu...i see dead people xD ab...
1042668,4,1957229112,Thu May 28 23:53:20 PDT 2009,NO_QUERY,000catnap000,awake since six o'clock... i had two hours sle...
1177063,4,1981357110,Sun May 31 08:59:07 PDT 2009,NO_QUERY,000catnap000,this is one of the cartoons i used to watch to...
1422100,4,2058432528,Sat Jun 06 14:55:54 PDT 2009,NO_QUERY,000catnap000,@Wumbologist hmmm...why not
1425085,4,2058981540,Sat Jun 06 16:00:48 PDT 2009,NO_QUERY,000catnap000,can't get enough of it http://bit.ly/134sSZ


In [3]:
tweets = {}
for user in tqdm(g):
    tweets[user] = df.iloc[g[user]][5].values

100%|████████████████████████████████████████████████████████████████████████| 659775/659775 [07:24<00:00, 1483.84it/s]


In [4]:
def clean(doc):
    # input: string
    # output: string
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.corpus import stopwords 
    import string
    lemma = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude or ch == "!")
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()).split()
    link_free = [word for word in normalized if 'http' not in word]
    cleaned = " ".join(word for word in link_free)
    
    return cleaned

In [5]:
# getting sentiment of sentence
import nltk
import string as st
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
def sentiment(tweets):
    # input: list of strings
    # output: list of ints
    sentiments = []
    for tweet in tweets:
        tweet = clean(tweet)
        score = sid.polarity_scores(tweet)
        sentiments.append(score['compound'])
    return sentiments

C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
sentiments = {}
overall = 0
for user in tqdm(tweets):
    sentiments[user] = sentiment(tweets[user])

100%|█████████████████████████████████████████████████████████████████████████| 659775/659775 [31:34<00:00, 348.23it/s]


In [122]:
import csv
with open('random_tweet_sentiments.tsv', 'w',encoding='utf-8') as f:
    f.write('user\ttweets\tsentiments\n')
    # grab the first 100k to make file smaller (observable file size limit)
    for key, value in tqdm(list(tweets.items())[:100000]):
        skip_indices = []
        f.write(key+'\t[')
        adskfljh = ''
        for i,v in enumerate(value):
            if ('\t' not in v):
                adskfljh += ('\"'+v+'\"'+',')
            else:
                skip_indices.append(i)
        if len(adskfljh) > 0 and adskfljh[-1] == ',':
            adskfljh = adskfljh[:-1]
        f.write(adskfljh)
        write_dis=[]
        for igiveup in range(len(sentiments[key])):
            if igiveup not in skip_indices:
                write_dis.append(sentiments[key][igiveup])
        f.write(']\t'+str(write_dis) + '\n')

100%|██████████████████████████████████████████████████████████████████████| 100000/100000 [00:00<00:00, 130380.05it/s]
